In [1]:
# yelp scraping project
import requests
import json
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import re

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 2500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [3]:
# hard code this section to scrape multiple pages for reviews
main = 'https://www.yelp.com/biz/fresh-cuts-daly-city'
url = [main,
       main + '?start=20',
       main + '?start=40',
       main + '?start=60',]
#        main + '?start=80',
#        main + '?start=100',]
#        main + '?start=120',
#        main + '?start=140',
#        main + '?start=160',
#        main + '?start=180',
#        main + '?start=200',
#        main + '?start=220',
#        main + '?start=240',
#        main + '?start=260',
#        main + '?start=280',
#        main + '?start=300',
#        main + '?start=320',]
#        main + '?start=340',
#        main + '?start=360',
#        main + '?start=380',]

In [4]:
for i in url:
    print (i)

https://www.yelp.com/biz/fresh-cuts-daly-city
https://www.yelp.com/biz/fresh-cuts-daly-city?start=20
https://www.yelp.com/biz/fresh-cuts-daly-city?start=40
https://www.yelp.com/biz/fresh-cuts-daly-city?start=60


# Start Review Section

In [5]:
reviews = []
star_rating = []

In [6]:
def get_review_content(url):
    for i in url:
        response = requests.get(i)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        rc_unstripped = soup.find_all(attrs={'class': 'review-content'})
        rc_length = (len(rc_unstripped))
        
        for i in range (0, rc_length):
            reviews.append(rc_unstripped[i].text.strip()) 
        
        for rating in soup.select(".rating-large"):
            star_rating.append(rating.get("title"))

In [7]:
# Run the function
get_review_content(url)

In [8]:
print(len(reviews))
print(len(star_rating))

62
68


In [9]:
print(reviews[0])
print(star_rating[0])

3/9/2018
    

I have been going here for just over a year now. Have found few shops and people that can not just cut my hair but my two sons. I look for convenience but quality and Fresh Cuts And Omar are that for me! He is able to get me in most the times I need and gives 3 of us quality cuts!
5.0 star rating


In [10]:
# convert reviews to pd series
reviews_s = pd.Series(reviews)
reviews_s.tail()

57    1/26/2011\n    \n\nBest haircuts in the bay ar...
58    4/3/2010\n    \n\n5 STARS for the Shop-nice cl...
59    5/13/2010\n    \n\nBeen to alot of barber shop...
60    12/2/2010\n    \n\nit  dont even matter who yo...
61    9/20/2009\n    \n\n\n\n\n\n\n\n\nFirst to Revi...
dtype: object

In [11]:
# convert ratings to pd series
ratings_s = pd.Series(star_rating)
ratings_s.tail()

63    5.0 star rating
64    4.0 star rating
65    5.0 star rating
66    5.0 star rating
67    4.0 star rating
dtype: object

In [12]:
# create a pandas df from reviews
data = {'Reviews' : reviews_s,
       'StarRating': ratings_s}

df = pd.DataFrame.from_dict(data)
df = df[['Reviews', 'StarRating']]    

In [13]:
df.head()
df.tail()
df.shape

,Reviews,StarRating
0,3/9/2018\n \n\nI have been going here for j...,5.0 star rating
1,3/8/2018\n \n\nI made an appointment with O...,1.0 star rating
2,3/11/2018\n \n\nIf you're gonna take appoin...,1.0 star rating
3,3/9/2018\n \n\nLast second haircut for a we...,5.0 star rating
4,11/25/2017\n \n\nBarber: OmarService: FHair...,1.0 star rating


,Reviews,StarRating
63,NaN,5.0 star rating
64,NaN,4.0 star rating
65,NaN,5.0 star rating
66,NaN,5.0 star rating
67,NaN,4.0 star rating


(68, 2)

In [14]:
# split date and reviews
df['Date'] = df.Reviews.str.split('\n').str.get(0)
df['Review'] = df.Reviews.str.split('\n\n').str.get(1)

In [15]:
# drop Reviews column
df.drop('Reviews', axis=1, inplace=True)

In [16]:
df.head()

,StarRating,Date,Review
0,5.0 star rating,3/9/2018,I have been going here for just over a year no...
1,1.0 star rating,3/8/2018,I made an appointment with Omar. When I arriv...
2,1.0 star rating,3/11/2018,"If you're gonna take appointments, keep them. ..."
3,5.0 star rating,3/9/2018,Last second haircut for a wedding Omar saved m...
4,1.0 star rating,11/25/2017,Barber: OmarService: FHaircut: FOverall Grade:...


In [17]:
# delete NaN rows
df2 = df.replace(r"", np.nan)
df2.tail()

,StarRating,Date,Review
63,5.0 star rating,NaN,NaN
64,4.0 star rating,NaN,NaN
65,5.0 star rating,NaN,NaN
66,5.0 star rating,NaN,NaN
67,4.0 star rating,NaN,NaN


In [18]:
# drop NaN rows
df3 = df2.dropna()

# reset index
df3.reset_index(drop=True, inplace=True)

df3.head()
df3.shape

,StarRating,Date,Review
0,5.0 star rating,3/9/2018,I have been going here for just over a year no...
1,1.0 star rating,3/8/2018,I made an appointment with Omar. When I arriv...
2,1.0 star rating,3/11/2018,"If you're gonna take appointments, keep them. ..."
3,5.0 star rating,3/9/2018,Last second haircut for a wedding Omar saved m...
4,1.0 star rating,11/25/2017,Barber: OmarService: FHaircut: FOverall Grade:...


(45, 3)

In [19]:
# checks to see if there are any null values
df3.isnull().sum()

StarRating    0
Date          0
Review        0
dtype: int64

# Pandas DF and Sentiment Section

In [20]:
from textblob import TextBlob

In [21]:
# convert Review column in df3 to list
review_list = df3['Review'].tolist()
review_list[0]

'I have been going here for just over a year now. Have found few shops and people that can not just cut my hair but my two sons. I look for convenience but quality and Fresh Cuts And Omar are that for me! He is able to get me in most the times I need and gives 3 of us quality cuts!'

In [22]:
len(review_list)

45

In [23]:
sentiment_score = []
# iterate through review list
for r in review_list:
    sentiment = TextBlob(r)
    sentiment_score.append(sentiment.sentiment.polarity)

In [24]:
# convert to series
sentiment_score_convert = pd.Series(sentiment_score)

In [25]:
# add sentiment score to sentiment_df
df3.insert(loc=1, column='SentimentScore', value=sentiment_score_convert)

In [26]:
df3.head()

,StarRating,SentimentScore,Date,Review
0,5.0 star rating,0.325000,3/9/2018,I have been going here for just over a year no...
1,1.0 star rating,-0.001491,3/8/2018,I made an appointment with Omar. When I arriv...
2,1.0 star rating,0.162500,3/11/2018,"If you're gonna take appointments, keep them. ..."
3,5.0 star rating,0.200000,3/9/2018,Last second haircut for a wedding Omar saved m...
4,1.0 star rating,0.019002,11/25/2017,Barber: OmarService: FHaircut: FOverall Grade:...


In [27]:
sentiment = []
# iterate through sentiment_score to assign a pos or neg val
for s in sentiment_score:
    if s > 0:
        sentiment.append('Positive')
    else:
        sentiment.append('Negative')

In [28]:
print(len(sentiment_score))
print(len(sentiment))

45
45


In [29]:
# check for null values
df3.isnull().sum()

StarRating        0
SentimentScore    0
Date              0
Review            0
dtype: int64

In [30]:
# convert sentiment to a pandas series
sentiment_2 = pd.Series(sentiment)

In [31]:
# add sentiment score to sentiment_df
df3.insert(loc=2, column='Sentiment', value=sentiment_2)

In [32]:
df3.head()

,StarRating,SentimentScore,Sentiment,Date,Review
0,5.0 star rating,0.325000,Positive,3/9/2018,I have been going here for just over a year no...
1,1.0 star rating,-0.001491,Negative,3/8/2018,I made an appointment with Omar. When I arriv...
2,1.0 star rating,0.162500,Positive,3/11/2018,"If you're gonna take appointments, keep them. ..."
3,5.0 star rating,0.200000,Positive,3/9/2018,Last second haircut for a wedding Omar saved m...
4,1.0 star rating,0.019002,Positive,11/25/2017,Barber: OmarService: FHaircut: FOverall Grade:...


In [33]:
# check value counts for Positive and Negative
df3.Sentiment.value_counts()

Positive    38
Negative     7
Name: Sentiment, dtype: int64

In [34]:
# convert Sentiment label to a numerical value
df3['SentimentNum'] = df3.Sentiment.map({'Positive': 0,
                                                 'Negative': 1})

In [35]:
df3.head()
df3.tail()
df3.shape

,StarRating,SentimentScore,Sentiment,Date,Review,SentimentNum
0,5.0 star rating,0.325000,Positive,3/9/2018,I have been going here for just over a year no...,0
1,1.0 star rating,-0.001491,Negative,3/8/2018,I made an appointment with Omar. When I arriv...,1
2,1.0 star rating,0.162500,Positive,3/11/2018,"If you're gonna take appointments, keep them. ...",0
3,5.0 star rating,0.200000,Positive,3/9/2018,Last second haircut for a wedding Omar saved m...,0
4,1.0 star rating,0.019002,Positive,11/25/2017,Barber: OmarService: FHaircut: FOverall Grade:...,0


,StarRating,SentimentScore,Sentiment,Date,Review,SentimentNum
40,5.0 star rating,0.125924,Positive,3/22/2013,"Out of every barbershop I've ever been too, th...",0
41,1.0 star rating,0.011843,Positive,1/26/2011,"Best haircuts in the bay area, ive lived all o...",0
42,1.0 star rating,0.128742,Positive,4/3/2010,5 STARS for the Shop-nice clean look0 STARS (i...,0
43,2.0 star rating,0.257407,Positive,5/13/2010,Been to alot of barber shops in the city and t...,0
44,5.0 star rating,0.300000,Positive,12/2/2010,it dont even matter who you go to at this sho...,0


(45, 6)

In [36]:
df3.SentimentNum.value_counts()

0    38
1     7
Name: SentimentNum, dtype: int64

In [37]:
# extract business name from "main" url
bizname_split = main.split('https://www.yelp.com/biz/')
bizname = bizname_split[1]
bizname

'fresh-cuts-daly-city'

In [38]:
# split Star Rating and create a new column
df3['Rating'] = df3.StarRating.str.split('star rating').str.get(0)

In [39]:
# reorder columns
df3 = df3[['Date', 'SentimentScore', 'Sentiment', 'SentimentNum', 'StarRating', 'Rating','Review']]
df3.head()

,Date,SentimentScore,Sentiment,SentimentNum,StarRating,Rating,Review
0,3/9/2018,0.325000,Positive,0,5.0 star rating,5.0,I have been going here for just over a year no...
1,3/8/2018,-0.001491,Negative,1,1.0 star rating,1.0,I made an appointment with Omar. When I arriv...
2,3/11/2018,0.162500,Positive,0,1.0 star rating,1.0,"If you're gonna take appointments, keep them. ..."
3,3/9/2018,0.200000,Positive,0,5.0 star rating,5.0,Last second haircut for a wedding Omar saved m...
4,11/25/2017,0.019002,Positive,0,1.0 star rating,1.0,Barber: OmarService: FHaircut: FOverall Grade:...


In [40]:
# save csv to reviews folder to use for NLP
df3.to_csv(f'reviews/{bizname}.csv', index=False)

# Section for NLTK

### Import saved csvs from reviews directory

In [ ]:
# Define X and y for use with COUNTVECTORIZE
X = df3.Review
y = df3.SentimentNum
print(X.shape)
print(y.shape)

### section to concat metadata dfs

In [ ]:
df1 = pd.read_csv('data/yelp_Barbers90742.csv')

In [ ]:
df2 = pd.read_csv('data/yelp_Barber92620.csv')

In [ ]:
df3 = pd.read_csv('data/yelp_Barbers94016.csv')

In [ ]:
df_merge = pd.concat([df1, df2, df3])

In [ ]:
df_merge.reset_index(drop=True, inplace=True)

In [ ]:
df_merge.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df_merge